# Thai Election Form Extractor - BigQuery + Google Drive Edition

Extract structured data from election form PDFs stored in **Google Drive** using **BigQuery** to find files.

**Key Advantages:**
- 🔍 Query BigQuery to find PDF files
- 📁 Direct access to Google Drive files (no download needed!)
- 🚀 Uses Gemini's External URLs file input method
- 🤖 Structured output with Pydantic schema validation

**Reference:** [Gemini File Input Methods](https://ai.google.dev/gemini-api/docs/file-input-methods)

## 1. Setup and Dependencies

In [1]:
# Install required packages (run once)
!pip install -q google-cloud-bigquery google-genai pydantic pandas


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
import os
from typing import Optional

from google.cloud import bigquery
from google import genai
from google.genai import types
from pydantic import BaseModel, Field
from IPython.display import display, HTML
import pandas as pd

## 2. Configuration

In [ ]:
# Google Cloud Configuration
GOOGLE_CLOUD_PROJECT = os.getenv("GOOGLE_CLOUD_PROJECT", "YOUR_PROJECT_ID")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "")

# Model Configuration
MODEL_NAME = "gemini-exp-1206"  # or "gemini-2.0-flash-exp", "gemini-1.5-pro-002"

# BigQuery Configuration
BQ_TABLE = "sourceinth.vote69_ect.raw_files"

# Verify configuration
print(f"✅ Configuration")
print(f"   Project: {GOOGLE_CLOUD_PROJECT}")
print(f"   API Key: {'*' * 20 + GEMINI_API_KEY[-8:] if GEMINI_API_KEY and len(GEMINI_API_KEY) > 8 else 'NOT SET ⚠️'}")
print(f"   Model: {MODEL_NAME}")
print(f"   BigQuery Table: {BQ_TABLE}")

# Warn if API key is not set
if not GEMINI_API_KEY:
    print("\n⚠️  WARNING: GEMINI_API_KEY is not set!")
    print("   Set it with: export GEMINI_API_KEY='your-key-here'")
    print("   Or create a .env file with GEMINI_API_KEY=your-key-here")

## 3. Pydantic Schema (Same as Backend)

In [4]:
class NumberTextPair(BaseModel):
    """Thai document number representation (both Arabic numeral and Thai text)."""
    arabic: int = Field(..., description="Arabic numeral (e.g., 120)")
    thai_text: Optional[str] = Field(None, description="Thai text (e.g., 'หนึ่งร้อยยี่สิบ')")


class FormInfo(BaseModel):
    """Header information identifying the polling station."""
    form_type: Optional[str] = Field(None, description="Constituency or PartyList")
    set_number: Optional[str] = Field(None, description="Set number (ชุดที่)")  # NEW
    date: Optional[str] = Field(None, description="Date of election")
    province: Optional[str] = Field(None, description="Province name")
    constituency_number: Optional[str] = Field(None, description="Constituency number")
    district: str = Field(..., description="District name")
    sub_district: Optional[str] = Field(None, description="Sub-district name")
    polling_station_number: str = Field(..., description="Polling station number")
    village_moo: Optional[str] = Field(None, description="Village number (หมู่ที่)")  # NEW


class VoterStatistics(BaseModel):
    """Voter statistics (Section 1)."""
    eligible_voters: Optional[NumberTextPair] = Field(None, description="Total eligible voters")
    present_voters: Optional[NumberTextPair] = Field(None, description="Voters who showed up")


class BallotStatistics(BaseModel):
    """Ballot accounting statistics (Section 2)."""
    ballots_allocated: Optional[NumberTextPair] = Field(None, description="Allocated ballots")
    ballots_used: Optional[NumberTextPair] = Field(None, description="Used ballots")
    good_ballots: Optional[NumberTextPair] = Field(None, description="Valid ballots")
    bad_ballots: Optional[NumberTextPair] = Field(None, description="Invalid ballots")
    no_vote_ballots: Optional[NumberTextPair] = Field(None, description="No vote ballots")
    ballots_remaining: Optional[NumberTextPair] = Field(None, description="Remaining ballots")


class VoteResult(BaseModel):
    """Individual vote result."""
    number: int = Field(..., description="Candidate/Party number")
    candidate_name: Optional[str] = Field(None, description="Candidate name (Constituency only)")
    party_name: Optional[str] = Field(None, description="Party name")
    vote_count: NumberTextPair = Field(..., description="Vote count (number + text)")


class Official(BaseModel):
    """Committee member/official."""
    name: str = Field(..., description="Full name of official")
    position: str = Field(..., description="Position/role (e.g., ประธาน, กรรมการ)")


class ElectionFormData(BaseModel):
    """Complete election form extraction result."""
    form_info: FormInfo
    voter_statistics: Optional[VoterStatistics] = None
    ballot_statistics: Optional[BallotStatistics] = None
    vote_results: list[VoteResult] = Field(default_factory=list)
    total_votes_recorded: Optional[NumberTextPair] = Field(
        None, 
        description="Total vote count from table footer"
    )  # NEW
    officials: Optional[list[Official]] = Field(
        None,
        description="Committee members who signed the form"
    )  # NEW


print("✅ Enhanced Pydantic models defined with NumberTextPair")

✅ Enhanced Pydantic models defined with NumberTextPair


## 4. Gemini Schema for Structured Output

In [5]:
# Enhanced schema for Gemini structured output with NumberTextPair
ELECTION_DATA_SCHEMA = {
    "type": "ARRAY",
    "description": "List of election reports found in the PDF",
    "items": {
        "type": "OBJECT",
        "properties": {
            "form_info": {
                "type": "OBJECT",
                "description": "Header information",
                "properties": {
                    "form_type": {
                        "type": "STRING",
                        "enum": ["Constituency", "PartyList"],
                        "description": "Form type: Constituency (candidates) or PartyList (parties only)"
                    },
                    "set_number": {"type": "STRING", "description": "Set number (ชุดที่)"},
                    "date": {"type": "STRING", "description": "Date of election"},
                    "province": {"type": "STRING", "description": "Province name"},
                    "constituency_number": {"type": "STRING", "description": "Constituency number"},
                    "district": {"type": "STRING", "description": "District name"},
                    "sub_district": {"type": "STRING", "description": "Sub-district name"},
                    "polling_station_number": {"type": "STRING", "description": "Polling station number"},
                    "village_moo": {"type": "STRING", "description": "Village number (หมู่ที่)"},
                },
                "required": ["form_type", "province", "district", "polling_station_number"],
            },
            "voter_statistics": {
                "type": "OBJECT",
                "description": "Section 1: Voter statistics",
                "properties": {
                    "eligible_voters": {
                        "type": "OBJECT",
                        "description": "1.1 Total eligible voters",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        }
                    },
                    "present_voters": {
                        "type": "OBJECT",
                        "description": "1.2 Voters who showed up",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        }
                    }
                }
            },
            "ballot_statistics": {
                "type": "OBJECT",
                "description": "Section 2: Ballot accounting",
                "properties": {
                    "ballots_allocated": {
                        "type": "OBJECT",
                        "description": "2.1 Allocated ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        }
                    },
                    "ballots_used": {
                        "type": "OBJECT",
                        "description": "2.2 Used ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        },
                        "required": ["arabic"]
                    },
                    "good_ballots": {
                        "type": "OBJECT",
                        "description": "2.2.1 Valid ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        },
                        "required": ["arabic"]
                    },
                    "bad_ballots": {
                        "type": "OBJECT",
                        "description": "2.2.2 Invalid ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        },
                        "required": ["arabic"]
                    },
                    "no_vote_ballots": {
                        "type": "OBJECT",
                        "description": "2.2.3 No vote ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        },
                        "required": ["arabic"]
                    },
                    "ballots_remaining": {
                        "type": "OBJECT",
                        "description": "2.3 Remaining ballots",
                        "properties": {
                            "arabic": {"type": "INTEGER"},
                            "thai_text": {"type": "STRING"}
                        }
                    }
                }
            },
            "vote_results": {
                "type": "ARRAY",
                "description": "Section 3: Vote counts for all candidates/parties",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "number": {"type": "INTEGER", "description": "Candidate/Party number"},
                        "candidate_name": {
                            "type": "STRING",
                            "description": "Candidate name (for Constituency forms only)"
                        },
                        "party_name": {"type": "STRING", "description": "Party name"},
                        "vote_count": {
                            "type": "OBJECT",
                            "description": "Vote count (both number and Thai text)",
                            "properties": {
                                "arabic": {"type": "INTEGER"},
                                "thai_text": {"type": "STRING"}
                            },
                            "required": ["arabic"]
                        }
                    },
                    "required": ["number", "vote_count"]
                },
            },
            "total_votes_recorded": {
                "type": "OBJECT",
                "description": "Total vote count from bottom of table (for validation)",
                "properties": {
                    "arabic": {"type": "INTEGER"},
                    "thai_text": {"type": "STRING"}
                }
            },
            "officials": {
                "type": "ARRAY",
                "description": "Committee members who signed the form",
                "items": {
                    "type": "OBJECT",
                    "properties": {
                        "name": {"type": "STRING", "description": "Full name"},
                        "position": {"type": "STRING", "description": "Position (ประธาน, กรรมการ, etc.)"}
                    },
                    "required": ["name", "position"]
                }
            }
        },
        "required": ["form_info", "vote_results"],
    },
}

print("✅ Enhanced Gemini schema defined with NumberTextPair")

✅ Enhanced Gemini schema defined with NumberTextPair


## 5. Initialize Clients

In [ ]:
# Initialize BigQuery client
bq_client = bigquery.Client(project=GOOGLE_CLOUD_PROJECT)
print("✅ BigQuery client initialized")

# Initialize Gemini client with API key
if not GEMINI_API_KEY:
    raise ValueError(
        "GEMINI_API_KEY is required! Set it with:\n"
        "  export GEMINI_API_KEY='your-key-here'\n"
        "  or create a .env file"
    )

gemini_client = genai.Client(
    api_key=GEMINI_API_KEY,
)
print("✅ Gemini client initialized (using API key)")
print(f"   Ready to use {MODEL_NAME}")

## 6. Query BigQuery for PDF Files

In [7]:
def query_pdf_files(
    limit: int = 10,
    province: Optional[str] = None,
    min_size_kb: float = 50.0,
    max_size_mb: Optional[float] = 50.0
) -> list[dict]:
    """
    Query BigQuery for PDF files.
    
    Args:
        limit: Maximum number of files to return
        province: Filter by province name (optional)
        min_size_kb: Minimum file size in KB (default: 50 KB to exclude corrupted files)
        max_size_mb: Maximum file size in MB (optional)
    
    Returns:
        List of file metadata dicts
    """
    # Build query
    conditions = ["mime_type = 'application/pdf'"]
    
    # Add minimum size filter (exclude very small/corrupted files)
    min_bytes = int(min_size_kb * 1024)
    conditions.append(f"size >= {min_bytes}")
    
    if province:
        conditions.append(f"province_name = '{province}'")
    
    if max_size_mb:
        max_bytes = int(max_size_mb * 1024 * 1024)
        conditions.append(f"size <= {max_bytes}")
    
    where_clause = " AND ".join(conditions)
    
    query = f"""
    SELECT 
        file_id, 
        path,
        mime_type, 
        folder_id, 
        province_name,
        size,
        mod_time
    FROM `{BQ_TABLE}`
    WHERE {where_clause}
    ORDER BY size ASC
    LIMIT {limit}
    """
    
    print(f"🔍 Querying BigQuery...")
    print(f"   Filters: {where_clause}")
    
    # Execute query
    query_job = bq_client.query(query)
    results = query_job.result()
    
    # Convert to list
    files = []
    for row in results:
        files.append({
            "file_id": row.file_id,
            "path": row.path,
            "mime_type": row.mime_type,
            "folder_id": row.folder_id,
            "province_name": row.province_name,
            "size": row.size,
            "size_mb": row.size / (1024 * 1024) if row.size else 0,
            "size_kb": row.size / 1024 if row.size else 0,
            "mod_time": row.mod_time,
        })
    
    print(f"✅ Found {len(files)} file(s)")
    print(f"   Size range: {files[0]['size_kb']:.1f} KB - {files[-1]['size_kb']:.1f} KB" if files else "")
    return files


# Query for files (min 50 KB, max 50 MB)
pdf_files = query_pdf_files(limit=10, min_size_kb=50.0, max_size_mb=50.0)

# Display as DataFrame
if pdf_files:
    df = pd.DataFrame(pdf_files)
    display(df[['province_name', 'path', 'size_mb', 'file_id']].head())

🔍 Querying BigQuery...
   Filters: mime_type = 'application/pdf' AND size >= 51200 AND size <= 52428800
✅ Found 10 file(s)
   Size range: 50.0 KB - 50.4 KB


,province_name,path,size_mb,file_id
0,พิจิตร,เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ทต.โพธิ...,0.048851,1_j0DNaqCXIkEk0MK3y0J1eCN3hUOCXeF
1,พิจิตร,เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ตำบลไผ่...,0.048915,1gDxp58u2W14uhdb6NpRDqxl1d7aa2WFy
2,พิจิตร,เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ตำบลไผ่...,0.048917,1a5jF1Oyv3UEatBq1MT1ga8kS10uR19-c
3,พิจิตร,เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ตำบลดงเ...,0.048927,1tzz6gMXk1n3pQtreQWMIU2xlkncg-g_r
4,พิจิตร,เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ทต.โพธิ...,0.048993,1-MsML3nSXUrscvmzdZb7R4yTkZcuTc5m


## 7. Select Test File

In [8]:
# Select first file for testing
test_file = pdf_files[0]

print("📄 Selected Test File:")
print("=" * 80)
print(f"Province: {test_file['province_name']}")
print(f"Path: {test_file['path']}")
print(f"File ID: {test_file['file_id']}")
print(f"Size: {test_file['size_mb']:.2f} MB")
print(f"Modified: {test_file['mod_time']}")
print()

# Construct Google Drive URI
drive_uri = f"https://drive.google.com/uc?export=download&id={test_file['file_id']}"
print(f"📁 Google Drive URI:")
print(drive_uri)

# Store for later use
TEST_FILE_ID = test_file['file_id']
TEST_DRIVE_URI = drive_uri

📄 Selected Test File:
Province: พิจิตร
Path: เขตเลือกตั้งที่ 3/อำเภอโพธิ์ประทับช้าง/ทต.โพธิ์ประทับช้าง/หน่วยเลือกตั้งที่ 9/สส5ทับ18 น_09.pdf
File ID: 1_j0DNaqCXIkEk0MK3y0J1eCN3hUOCXeF
Size: 0.05 MB
Modified: 2026-02-10T03:26:51.000Z

📁 Google Drive URI:
https://drive.google.com/uc?export=download&id=1_j0DNaqCXIkEk0MK3y0J1eCN3hUOCXeF


## 8. Extract Data Using Gemini with Google Drive URL

**Key Advantage:** No need to download the PDF locally! Gemini can access it directly from Google Drive.

In [ ]:
def extract_from_drive_url(
    drive_uri: str,
    model: str = MODEL_NAME,
    temperature: float = 0.0,
    max_tokens: int = 8192,
) -> dict:
    """
    Extract vote data from a PDF file stored in Google Drive.
    
    Args:
        drive_uri: Google Drive file URI (https://drive.google.com/uc?export=download&id=...)
        model: Gemini model name
        temperature: Sampling temperature
        max_tokens: Maximum output tokens
    
    Returns:
        Extracted data as dictionary
    """
    print(f"🤖 Extracting with {model}...")
    print(f"   Using Google Drive URI (External URL method)")
    
    # Create file part from URI
    # Gemini will download and process the file directly
    file_part = types.Part.from_uri(
        file_uri=drive_uri,
        mime_type="application/pdf"
    )
    
    # Enhanced extraction prompt
    prompt = """
    You are an expert data entry assistant for Thai Election documents (Form S.S. 5/18).
    
    CRITICAL INSTRUCTIONS:
    
    1. **Analyze all pages** of this PDF document carefully.
    
    2. **Extract BOTH number formats** for all numerical values:
       - Arabic numerals (e.g., 120)
       - Thai text (e.g., "หนึ่งร้อยยี่สิบ")
       This applies to: voter statistics, ballot statistics, vote counts, and total votes.
    
    3. **Header Information** (usually on first page):
       - Form type: "Constituency" (แบบแบ่งเขต) or "PartyList" (บัญชีรายชื่อ)
       - Set number (ชุดที่) if present
       - Date, Province, District, Sub-district
       - Polling station number (หน่วยเลือกตั้งที่)
       - Village number (หมู่ที่) if present
    
    4. **Section 1 - Voter Statistics:**
       - 1.1 Eligible voters (ผู้มีสิทธิเลือกตั้งตามบัญชี)
       - 1.2 Present voters (ผู้มาแสดงตน)
       Extract both arabic and thai_text for each.
    
    5. **Section 2 - Ballot Statistics:**
       - 2.1 Allocated ballots (บัตรที่ได้รับจัดสรร)
       - 2.2 Used ballots (บัตรที่ใช้)
       - 2.2.1 Valid ballots (บัตรดี)
       - 2.2.2 Invalid ballots (บัตรเสีย)
       - 2.2.3 No vote ballots (ไม่เลือก)
       - 2.3 Remaining ballots (บัตรเหลือ)
       Extract both arabic and thai_text for each.
    
    6. **Section 3 - Vote Results Table:**
       - Consolidate all pages (table often spans multiple pages)
       - For each entry: number, candidate name (if Constituency), party name, vote count
       - Extract vote_count as {arabic: int, thai_text: str}
    
    7. **Total Votes Recorded:**
       - Look for "รวม" (total) at the bottom of the vote results table
       - Extract both arabic and thai_text
    
    8. **Officials (Committee Members):**
       - Extract names and positions from signature section
       - Common positions: ประธาน (Chair), กรรมการ (Member), เลขานุการ (Secretary)
    
    9. **Validation:**
       - ballots_used.arabic = good_ballots.arabic + bad_ballots.arabic + no_vote_ballots.arabic
       - total_votes_recorded.arabic = sum of all vote_count.arabic
    """
    
    # Configure generation
    generation_config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ELECTION_DATA_SCHEMA,
        temperature=temperature,
        max_output_tokens=max_tokens,
        top_p=0.95,
        top_k=40,
        thinking_config=types.ThinkingConfig(
            thinking_level="LOW"
         ),
    )
    
    # Generate content
    print("   Sending request to Gemini...")
    response = gemini_client.models.generate_content(
        model=model,
        contents=[file_part, prompt],
        config=generation_config,
    )
    
    # Parse response
    result = json.loads(response.text)
    
    print(f"✅ Extraction complete!")
    print(f"   Extracted {len(result)} report(s)")
    
    return result


print("✅ Enhanced extraction function defined")

✅ Enhanced extraction function defined


## 9. Run Extraction

In [10]:
# Extract data from Google Drive PDF
result = extract_from_drive_url(
    drive_uri=TEST_DRIVE_URI,
    model=MODEL_NAME,
    temperature=0.0,
    max_tokens=32976,
)

🤖 Extracting with gemini-3-pro-preview...
   Using Google Drive URI (External URL method)
   Sending request to Gemini...
✅ Extraction complete!
   Extracted 1 report(s)


## 10. Display Results

In [18]:
def get_number_value(num_obj) -> int:
    """Extract arabic number from NumberTextPair or plain int."""
    if isinstance(num_obj, dict):
        return num_obj.get('arabic', 0)
    elif isinstance(num_obj, int):
        return num_obj
    return 0


def get_thai_text(num_obj) -> str:
    """Extract Thai text from NumberTextPair or return empty string."""
    if isinstance(num_obj, dict):
        return num_obj.get('thai_text', '')
    return ''


def display_results(result: list[dict]):
    """Display enhanced extraction results with NumberTextPair support."""
    if not result:
        print("❌ No data extracted")
        return
    
    for idx, report in enumerate(result, 1):
        print(f"\n{'='*80}")
        print(f"REPORT #{idx}")
        print(f"{'='*80}")
        
        # Form Info
        form_info = report.get("form_info", {})
        print(f"\n📋 FORM INFORMATION")
        print(f"   Form Type: {form_info.get('form_type', 'N/A')}")
        
        # Show set_number if available
        if form_info.get('set_number'):
            print(f"   Set Number: {form_info.get('set_number')}")
        
        print(f"   Province: {form_info.get('province', 'N/A')}")
        print(f"   District: {form_info.get('district', 'N/A')}")
        
        if form_info.get('sub_district'):
            print(f"   Sub-district: {form_info.get('sub_district')}")
        
        print(f"   Station: {form_info.get('polling_station_number', 'N/A')}")
        
        # Show village_moo if available
        if form_info.get('village_moo'):
            print(f"   Village (หมู่): {form_info.get('village_moo')}")
        
        if form_info.get('date'):
            print(f"   Date: {form_info.get('date')}")
        
        # Voter Statistics
        voter_stats = report.get("voter_statistics")
        if voter_stats and (voter_stats.get("eligible_voters") or voter_stats.get("present_voters")):
            print(f"\n👥 VOTER STATISTICS")
            
            eligible = voter_stats.get("eligible_voters")
            if eligible:
                arabic = get_number_value(eligible)
                thai = get_thai_text(eligible)
                if thai:
                    print(f"   Eligible: {arabic:,} ({thai})")
                else:
                    print(f"   Eligible: {arabic:,}")
            
            present = voter_stats.get("present_voters")
            if present:
                arabic = get_number_value(present)
                thai = get_thai_text(present)
                if thai:
                    print(f"   Present: {arabic:,} ({thai})")
                else:
                    print(f"   Present: {arabic:,}")
        
        # Ballot Statistics
        ballot_stats = report.get("ballot_statistics")
        if ballot_stats:
            print(f"\n📦 BALLOT STATISTICS")
            
            # Extract values safely
            used = get_number_value(ballot_stats.get('ballots_used'))
            good = get_number_value(ballot_stats.get('good_ballots'))
            bad = get_number_value(ballot_stats.get('bad_ballots'))
            no_vote = get_number_value(ballot_stats.get('no_vote_ballots'))
            allocated = get_number_value(ballot_stats.get('ballots_allocated'))
            remaining = get_number_value(ballot_stats.get('ballots_remaining'))
            
            if allocated > 0:
                print(f"   Allocated: {allocated:,}")
            if used > 0:
                print(f"   Used: {used:,}")
            if good > 0:
                print(f"   - Good: {good:,}")
            if bad > 0:
                print(f"   - Bad: {bad:,}")
            if no_vote > 0:
                print(f"   - No Vote: {no_vote:,}")
            if remaining > 0:
                print(f"   Remaining: {remaining:,}")
            
            # Validation
            if used > 0 and (good > 0 or bad > 0 or no_vote > 0):
                expected = good + bad + no_vote
                if used == expected:
                    print(f"   ✅ Validation: PASSED ({used:,} = {expected:,})")
                else:
                    print(f"   ⚠️  Validation: FAILED ({used:,} ≠ {expected:,})")
        
        # Vote Results
        vote_results = report.get("vote_results", [])
        if vote_results:
            print(f"\n📊 VOTE RESULTS ({len(vote_results)} entries)")
            
            # Create DataFrame
            df_data = []
            for v in vote_results:
                vote_count_obj = v.get("vote_count")
                vote_arabic = get_number_value(vote_count_obj)
                vote_thai = get_thai_text(vote_count_obj)
                
                row = {
                    "#": v.get("number"),
                    "Candidate": v.get("candidate_name") or "-",
                    "Party": v.get("party_name") or "-",
                    "Votes": vote_arabic,
                }
                
                # Add Thai text column if any results have it
                if vote_thai:
                    row["Votes (Thai)"] = vote_thai[:30] + "..." if len(vote_thai) > 30 else vote_thai
                
                df_data.append(row)
            
            df = pd.DataFrame(df_data)
            display(df)
            
            # Calculate total
            total = df["Votes"].sum()
            print(f"\n   Calculated Total: {total:,}")
            
            # Show recorded total if available
            total_recorded = report.get("total_votes_recorded")
            if total_recorded:
                recorded_arabic = get_number_value(total_recorded)
                recorded_thai = get_thai_text(total_recorded)
                
                if recorded_thai:
                    print(f"   Recorded Total: {recorded_arabic:,} ({recorded_thai})")
                else:
                    print(f"   Recorded Total: {recorded_arabic:,}")
                
                # Validation
                if total == recorded_arabic:
                    print(f"   ✅ Total validation: PASSED")
                else:
                    print(f"   ⚠️  Total validation: FAILED ({total:,} ≠ {recorded_arabic:,})")
        
        # Officials
        officials = report.get("officials")
        if officials and len(officials) > 0:
            print(f"\n👔 COMMITTEE MEMBERS ({len(officials)} members)")
            for i, official in enumerate(officials[:10], 1):  # Show max 10
                name = official.get('name', 'N/A')
                position = official.get('position', 'N/A')
                print(f"   {i}. {name} - {position}")
            
            if len(officials) > 10:
                print(f"   ... and {len(officials) - 10} more")


# Display results
try:
    display_results(result)
except Exception as e:
    print(f"❌ Error displaying results: {e}")
    import traceback
    traceback.print_exc()
    
    # Show raw result for debugging
    print("\n🔍 Raw result (first 500 chars):")
    print(json.dumps(result, ensure_ascii=False, indent=2)[:500])


REPORT #1

📋 FORM INFORMATION
   Form Type: Constituency
   Province: พิจิตร
   District: โพธิ์ประทับช้าง
   Sub-district: วังจิก
   Station: 1
   Village (หมู่): 1
   Date: 14 พฤษภาคม 2566

👥 VOTER STATISTICS
   Eligible: 421 (สี่ร้อยยี่สิบเอ็ด)
   Present: 276 (สองร้อยเจ็ดสิบหก)

📦 BALLOT STATISTICS
   Allocated: 420
   Used: 276
   - Good: 264
   - Bad: 8
   - No Vote: 4
   Remaining: 144
   ✅ Validation: PASSED (276 = 276)

📊 VOTE RESULTS (7 entries)


,#,Candidate,Party,Votes,Votes (Thai)
0,1,นายธงชาติ วันศุกร์,กล้าธรรม,57,ห้าสิบเจ็ด
1,2,นายศิริวัฒน์ ขจรประศาสน์,ภูมิใจไทย,159,หนึ่งร้อยห้าสิบเก้า
2,3,นายวิทยา มาลา,เพื่อไทย,10,สิบ
3,4,นายศราวุธ ปลอดภัย,รวมไทยสร้างชาติ,0,ศูนย์
4,5,พันเอกมนู ชูจิตร,เสรีรวมไทย,1,หนึ่ง
5,6,นายสุรชาติ มุกดา,ประชาธิปัตย์,1,หนึ่ง
6,7,นายภัทรพงศ์ ภัทรประสิทธิ์,ประชาชน,36,สามสิบหก



   Calculated Total: 264
   Recorded Total: 264 (สองร้อยหกสิบสี่)
   ✅ Total validation: PASSED


In [12]:
def validate_extraction_enhanced(data: dict) -> tuple[bool, list[str]]:
    """
    Enhanced validation with NumberTextPair support.
    
    Args:
        data: Extracted form data
    
    Returns:
        Tuple of (is_valid, list of error messages)
    """
    errors = []
    warnings = []
    
    # 1. Ballot statistics validation
    ballot_stats = data.get("ballot_statistics")
    if ballot_stats:
        used = get_number_value(ballot_stats.get("ballots_used"))
        good = get_number_value(ballot_stats.get("good_ballots"))
        bad = get_number_value(ballot_stats.get("bad_ballots"))
        no_vote = get_number_value(ballot_stats.get("no_vote_ballots"))
        
        expected_total = good + bad + no_vote
        
        if used != expected_total:
            errors.append(
                f"Ballot mismatch: ballots_used ({used:,}) != "
                f"good+bad+no_vote ({expected_total:,})"
            )
    
    # 2. Total votes validation (NEW!)
    vote_results = data.get("vote_results", [])
    total_recorded = data.get("total_votes_recorded")
    
    if vote_results and total_recorded:
        # Sum up all vote counts
        calculated_total = sum(get_number_value(v.get("vote_count")) for v in vote_results)
        recorded_total = get_number_value(total_recorded)
        
        if calculated_total != recorded_total:
            errors.append(
                f"Vote total mismatch: sum of votes ({calculated_total:,}) != "
                f"recorded total ({recorded_total:,})"
            )
    
    # 3. Voter statistics vs ballot statistics (NEW!)
    voter_stats = data.get("voter_statistics")
    if voter_stats and ballot_stats:
        present = get_number_value(voter_stats.get("present_voters"))
        used = get_number_value(ballot_stats.get("ballots_used"))
        
        # Present voters should roughly match ballots used (allow small discrepancy)
        discrepancy = abs(present - used)
        if discrepancy > 5:
            warnings.append(
                f"Voter count ({present:,}) differs from ballots used ({used:,}) by {discrepancy}"
            )
    
    # 4. Vote count non-negative check
    for i, result in enumerate(vote_results, 1):
        vote_count = get_number_value(result.get("vote_count"))
        if vote_count < 0:
            name = result.get("candidate_name") or result.get("party_name") or f"Entry #{i}"
            errors.append(f"Negative vote count for {name}: {vote_count}")
    
    # 5. Check for empty vote results
    if not vote_results:
        errors.append("No vote results extracted")
    
    # Display results
    print(f"\n{'='*80}")
    print(f"VALIDATION RESULTS")
    print(f"{'='*80}")
    
    if errors:
        print(f"\n❌ ERRORS ({len(errors)}):")
        for error in errors:
            print(f"   - {error}")
    
    if warnings:
        print(f"\n⚠️  WARNINGS ({len(warnings)}):")
        for warning in warnings:
            print(f"   - {warning}")
    
    if not errors and not warnings:
        print(f"\n✅ All validation checks PASSED!")
    elif not errors:
        print(f"\n✅ No errors, but {len(warnings)} warning(s)")
    
    return len(errors) == 0, errors


# Run validation on extracted data
is_valid, errors = validate_extraction_enhanced(result[0] if result else {})

print(f"\n{'='*80}")
print(f"Overall: {'✅ VALID' if is_valid else '❌ INVALID'}")


VALIDATION RESULTS

✅ All validation checks PASSED!

Overall: ✅ VALID


In [19]:
# Validate each report with enhanced Pydantic models
print("\n" + "="*80)
print("PYDANTIC MODEL VALIDATION")
print("="*80)

for idx, report_data in enumerate(result, 1):
    try:
        # Parse into Pydantic model
        form_data = ElectionFormData(**report_data)
        
        print(f"\n✅ Report #{idx} - Pydantic validation PASSED")
        print(f"   Form Type: {form_data.form_info.form_type}")
        print(f"   District: {form_data.form_info.district}")
        print(f"   Set Number: {form_data.form_info.set_number or 'N/A'}")
        print(f"   Village: {form_data.form_info.village_moo or 'N/A'}")
        
        # Show voter statistics if available
        if form_data.voter_statistics:
            if form_data.voter_statistics.eligible_voters:
                print(f"   Eligible Voters: {form_data.voter_statistics.eligible_voters.arabic:,}")
            if form_data.voter_statistics.present_voters:
                print(f"   Present Voters: {form_data.voter_statistics.present_voters.arabic:,}")
        
        # Show ballot statistics
        if form_data.ballot_statistics and form_data.ballot_statistics.ballots_used:
            print(f"   Ballots Used: {form_data.ballot_statistics.ballots_used.arabic:,}")
        
        # Show vote results count
        print(f"   Vote Results: {len(form_data.vote_results)} entries")
        
        # Show total votes if available
        if form_data.total_votes_recorded:
            print(f"   Total Votes: {form_data.total_votes_recorded.arabic:,}")
        
        # Show officials count if available
        if form_data.officials:
            print(f"   Officials: {len(form_data.officials)} members")
        
    except Exception as e:
        print(f"\n❌ Report #{idx} - Pydantic validation FAILED")
        print(f"   Error: {e}")
        
        # Show which field caused the error
        import traceback
        error_details = traceback.format_exc()
        if "Field required" in str(e):
            print(f"   Hint: Missing required field")
        elif "validation error" in str(e).lower():
            print(f"   Hint: Data type mismatch")
        
        # Show first few lines of error for debugging
        error_lines = error_details.split('\n')
        relevant_lines = [line for line in error_lines if 'Field' in line or 'validation' in line.lower()]
        if relevant_lines:
            print(f"   Details: {relevant_lines[0][:100]}")


PYDANTIC MODEL VALIDATION

✅ Report #1 - Pydantic validation PASSED
   Form Type: Constituency
   District: โพธิ์ประทับช้าง
   Set Number: N/A
   Village: 1
   Eligible Voters: 421
   Present Voters: 276
   Ballots Used: 276
   Vote Results: 7 entries
   Total Votes: 264


## 11. Validate with Pydantic

In [20]:
# Validate each report
for idx, report_data in enumerate(result, 1):
    try:
        form_data = ElectionFormData(**report_data)
        print(f"✅ Report #{idx} - Pydantic validation PASSED")
        print(f"   Form Type: {form_data.form_info.form_type}")
        print(f"   District: {form_data.form_info.district}")
        print(f"   Vote Results: {len(form_data.vote_results)} entries")
        print()
    except Exception as e:
        print(f"❌ Report #{idx} - Pydantic validation FAILED")
        print(f"   Error: {e}")
        print()

✅ Report #1 - Pydantic validation PASSED
   Form Type: Constituency
   District: โพธิ์ประทับช้าง
   Vote Results: 7 entries



## 12. Save Results

In [21]:
# Save to JSON
output_file = f"extracted_data_{test_file['file_id']}.json"

output_data = {
    "source_file": test_file,
    "extracted_data": result,
    "model": MODEL_NAME,
}

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"💾 Results saved to: {output_file}")

💾 Results saved to: extracted_data_1_j0DNaqCXIkEk0MK3y0J1eCN3hUOCXeF.json


In [22]:
from dataclasses import dataclass
from typing import Literal


@dataclass
class ValidationCheck:
    """Individual validation check result."""
    check_name: str
    status: Literal["pass", "fail", "warning", "skip"]
    message: str
    expected: Optional[int] = None
    actual: Optional[int] = None


@dataclass
class EvaluationResult:
    """Comprehensive evaluation result for a single report."""
    report_index: int
    is_valid: bool
    quality_score: float  # 0.0 to 1.0
    checks_passed: int
    checks_failed: int
    checks_warning: int
    checks_skipped: int
    total_checks: int
    validation_checks: list[ValidationCheck]
    errors: list[str]
    warnings: list[str]
    
    def to_dict(self) -> dict:
        """Convert to dictionary for JSON serialization."""
        return {
            "report_index": self.report_index,
            "is_valid": self.is_valid,
            "quality_score": self.quality_score,
            "checks_passed": self.checks_passed,
            "checks_failed": self.checks_failed,
            "checks_warning": self.checks_warning,
            "checks_skipped": self.checks_skipped,
            "total_checks": self.total_checks,
            "errors": self.errors,
            "warnings": self.warnings,
        }


def evaluate_extraction(data: dict, report_index: int = 0) -> EvaluationResult:
    """
    Comprehensive evaluation of extracted election form data.
    
    Performs multiple validation checks and computes quality score.
    
    Args:
        data: Extracted form data
        report_index: Index of the report (for tracking)
    
    Returns:
        EvaluationResult with detailed validation information
    """
    checks = []
    errors = []
    warnings = []
    
    # Check 1: Form info completeness
    form_info = data.get("form_info", {})
    required_fields = ["form_type", "province", "district", "polling_station_number"]
    missing_fields = [f for f in required_fields if not form_info.get(f)]
    
    if missing_fields:
        checks.append(ValidationCheck(
            check_name="form_info_complete",
            status="fail",
            message=f"Missing required fields: {', '.join(missing_fields)}"
        ))
        errors.append(f"Form info incomplete: {', '.join(missing_fields)}")
    else:
        checks.append(ValidationCheck(
            check_name="form_info_complete",
            status="pass",
            message="All required form info fields present"
        ))
    
    # Check 2: Ballot statistics validation
    ballot_stats = data.get("ballot_statistics")
    if ballot_stats:
        used = get_number_value(ballot_stats.get("ballots_used"))
        good = get_number_value(ballot_stats.get("good_ballots"))
        bad = get_number_value(ballot_stats.get("bad_ballots"))
        no_vote = get_number_value(ballot_stats.get("no_vote_ballots"))
        
        expected_total = good + bad + no_vote
        
        if used == expected_total:
            checks.append(ValidationCheck(
                check_name="ballot_accounting",
                status="pass",
                message=f"Ballot accounting correct: {used:,} = {expected_total:,}",
                expected=expected_total,
                actual=used
            ))
        else:
            checks.append(ValidationCheck(
                check_name="ballot_accounting",
                status="fail",
                message=f"Ballot mismatch: used={used:,}, expected={expected_total:,}",
                expected=expected_total,
                actual=used
            ))
            errors.append(f"Ballot mismatch: {used:,} ≠ {expected_total:,}")
    else:
        checks.append(ValidationCheck(
            check_name="ballot_accounting",
            status="skip",
            message="No ballot statistics found"
        ))
    
    # Check 3: Total votes validation (NEW!)
    vote_results = data.get("vote_results", [])
    total_recorded = data.get("total_votes_recorded")
    
    if vote_results and total_recorded:
        calculated_total = sum(get_number_value(v.get("vote_count")) for v in vote_results)
        recorded_total = get_number_value(total_recorded)
        
        if calculated_total == recorded_total:
            checks.append(ValidationCheck(
                check_name="total_votes",
                status="pass",
                message=f"Total votes match: {calculated_total:,} = {recorded_total:,}",
                expected=recorded_total,
                actual=calculated_total
            ))
        else:
            checks.append(ValidationCheck(
                check_name="total_votes",
                status="fail",
                message=f"Total mismatch: calculated={calculated_total:,}, recorded={recorded_total:,}",
                expected=recorded_total,
                actual=calculated_total
            ))
            errors.append(f"Total votes mismatch: {calculated_total:,} ≠ {recorded_total:,}")
    else:
        checks.append(ValidationCheck(
            check_name="total_votes",
            status="skip",
            message="Total votes not available for validation"
        ))
    
    # Check 4: Voter statistics validation (NEW!)
    voter_stats = data.get("voter_statistics")
    if voter_stats and ballot_stats:
        present = get_number_value(voter_stats.get("present_voters"))
        used = get_number_value(ballot_stats.get("ballots_used"))
        
        discrepancy = abs(present - used)
        
        if discrepancy <= 5:
            checks.append(ValidationCheck(
                check_name="voter_ballot_consistency",
                status="pass",
                message=f"Voter count consistent with ballots: {present:,} ≈ {used:,}",
                expected=used,
                actual=present
            ))
        else:
            checks.append(ValidationCheck(
                check_name="voter_ballot_consistency",
                status="warning",
                message=f"Discrepancy: present_voters={present:,}, ballots_used={used:,} (diff={discrepancy})",
                expected=used,
                actual=present
            ))
            warnings.append(f"Voter/ballot discrepancy: {discrepancy}")
    else:
        checks.append(ValidationCheck(
            check_name="voter_ballot_consistency",
            status="skip",
            message="Voter statistics not available"
        ))
    
    # Check 5: Vote results non-empty
    if not vote_results:
        checks.append(ValidationCheck(
            check_name="vote_results_exist",
            status="fail",
            message="No vote results extracted"
        ))
        errors.append("No vote results found")
    else:
        checks.append(ValidationCheck(
            check_name="vote_results_exist",
            status="pass",
            message=f"Vote results present: {len(vote_results)} entries"
        ))
    
    # Check 6: Non-negative vote counts
    negative_votes = []
    for i, result in enumerate(vote_results, 1):
        vote_count = get_number_value(result.get("vote_count"))
        if vote_count < 0:
            name = result.get("candidate_name") or result.get("party_name") or f"Entry #{i}"
            negative_votes.append(f"{name} ({vote_count})")
    
    if negative_votes:
        checks.append(ValidationCheck(
            check_name="non_negative_votes",
            status="fail",
            message=f"Found negative votes: {', '.join(negative_votes)}"
        ))
        errors.extend([f"Negative vote: {v}" for v in negative_votes])
    else:
        checks.append(ValidationCheck(
            check_name="non_negative_votes",
            status="pass",
            message="All vote counts are non-negative"
        ))
    
    # Check 7: Thai text extraction quality (NEW!)
    text_quality_score = 0.0
    text_fields_checked = 0
    
    # Check ballot statistics thai text
    if ballot_stats:
        for field in ["ballots_used", "good_ballots", "bad_ballots", "no_vote_ballots"]:
            field_obj = ballot_stats.get(field)
            if isinstance(field_obj, dict) and field_obj.get("thai_text"):
                text_quality_score += 1
            text_fields_checked += 1
    
    # Check vote results thai text
    for result in vote_results[:10]:  # Sample first 10
        vote_count_obj = result.get("vote_count")
        if isinstance(vote_count_obj, dict) and vote_count_obj.get("thai_text"):
            text_quality_score += 1
        text_fields_checked += 1
    
    thai_text_coverage = (text_quality_score / text_fields_checked * 100) if text_fields_checked > 0 else 0
    
    if thai_text_coverage >= 80:
        checks.append(ValidationCheck(
            check_name="thai_text_quality",
            status="pass",
            message=f"Thai text extraction: {thai_text_coverage:.1f}% coverage"
        ))
    elif thai_text_coverage >= 50:
        checks.append(ValidationCheck(
            check_name="thai_text_quality",
            status="warning",
            message=f"Thai text extraction: {thai_text_coverage:.1f}% coverage (low)"
        ))
        warnings.append(f"Low Thai text coverage: {thai_text_coverage:.1f}%")
    else:
        checks.append(ValidationCheck(
            check_name="thai_text_quality",
            status="fail",
            message=f"Thai text extraction: {thai_text_coverage:.1f}% coverage (very low)"
        ))
        errors.append(f"Very low Thai text coverage: {thai_text_coverage:.1f}%")
    
    # Calculate summary
    checks_passed = sum(1 for c in checks if c.status == "pass")
    checks_failed = sum(1 for c in checks if c.status == "fail")
    checks_warning = sum(1 for c in checks if c.status == "warning")
    checks_skipped = sum(1 for c in checks if c.status == "skip")
    total_checks = len(checks)
    
    # Quality score: (passed + 0.5*warning) / (passed + failed + warning)
    scoreable_checks = checks_passed + checks_failed + checks_warning
    quality_score = (checks_passed + 0.5 * checks_warning) / scoreable_checks if scoreable_checks > 0 else 0.0
    
    is_valid = checks_failed == 0
    
    return EvaluationResult(
        report_index=report_index,
        is_valid=is_valid,
        quality_score=quality_score,
        checks_passed=checks_passed,
        checks_failed=checks_failed,
        checks_warning=checks_warning,
        checks_skipped=checks_skipped,
        total_checks=total_checks,
        validation_checks=checks,
        errors=errors,
        warnings=warnings,
    )


def print_evaluation_summary(eval_result: EvaluationResult):
    """Print formatted evaluation summary."""
    print(f"\n{'='*80}")
    print(f"EVALUATION SUMMARY - Report #{eval_result.report_index + 1}")
    print(f"{'='*80}")
    print(f"\n📊 Overall Status: {'✅ VALID' if eval_result.is_valid else '❌ INVALID'}")
    print(f"📈 Quality Score: {eval_result.quality_score:.1%}")
    print(f"\n📋 Check Results:")
    print(f"   ✅ Passed: {eval_result.checks_passed}/{eval_result.total_checks}")
    print(f"   ❌ Failed: {eval_result.checks_failed}/{eval_result.total_checks}")
    print(f"   ⚠️  Warnings: {eval_result.checks_warning}/{eval_result.total_checks}")
    print(f"   ⏭️  Skipped: {eval_result.checks_skipped}/{eval_result.total_checks}")
    
    # Show detailed checks
    print(f"\n🔍 Detailed Checks:")
    for check in eval_result.validation_checks:
        status_icon = {
            "pass": "✅",
            "fail": "❌",
            "warning": "⚠️",
            "skip": "⏭️"
        }[check.status]
        
        print(f"   {status_icon} {check.check_name}: {check.message}")
        if check.expected is not None and check.actual is not None:
            print(f"      Expected: {check.expected:,}, Actual: {check.actual:,}")
    
    # Show errors
    if eval_result.errors:
        print(f"\n❌ ERRORS ({len(eval_result.errors)}):")
        for error in eval_result.errors:
            print(f"   - {error}")
    
    # Show warnings
    if eval_result.warnings:
        print(f"\n⚠️  WARNINGS ({len(eval_result.warnings)}):")
        for warning in eval_result.warnings:
            print(f"   - {warning}")


# Example: Evaluate the extracted result
if result:
    eval_result = evaluate_extraction(result[0], report_index=0)
    print_evaluation_summary(eval_result)
    
    # Show as dict (for saving)
    print(f"\n📦 Evaluation as JSON:")
    print(json.dumps(eval_result.to_dict(), indent=2, ensure_ascii=False))


EVALUATION SUMMARY - Report #1

📊 Overall Status: ✅ VALID
📈 Quality Score: 100.0%

📋 Check Results:
   ✅ Passed: 7/7
   ❌ Failed: 0/7
   ⚠️  Warnings: 0/7
   ⏭️  Skipped: 0/7

🔍 Detailed Checks:
   ✅ form_info_complete: All required form info fields present
   ✅ ballot_accounting: Ballot accounting correct: 276 = 276
      Expected: 276, Actual: 276
   ✅ total_votes: Total votes match: 264 = 264
      Expected: 264, Actual: 264
   ✅ voter_ballot_consistency: Voter count consistent with ballots: 276 ≈ 276
      Expected: 276, Actual: 276
   ✅ vote_results_exist: Vote results present: 7 entries
   ✅ non_negative_votes: All vote counts are non-negative
   ✅ thai_text_quality: Thai text extraction: 100.0% coverage

📦 Evaluation as JSON:
{
  "report_index": 0,
  "is_valid": true,
  "quality_score": 1.0,
  "checks_passed": 7,
  "checks_failed": 0,
  "checks_warning": 0,
  "checks_skipped": 0,
  "total_checks": 7,
  "errors": [],
  "warnings": []
}


## 13. Comprehensive Evaluation Function

Create a reusable evaluation function for quality assessment during batch processing.

## 14. Batch Processing with BigQuery

Process multiple PDF files in batch.

In [17]:
def batch_process_from_bigquery(
    limit: int = 5,
    province: Optional[str] = None,
    min_size_kb: float = 50.0,
    max_size_mb: float = 30.0,
    model: str = MODEL_NAME,
    run_evaluation: bool = True
) -> list[dict]:
    """
    Batch process PDF files from BigQuery with integrated evaluation.
    
    Args:
        limit: Maximum number of files to process
        province: Filter by province
        min_size_kb: Minimum file size in KB (default: 50 KB)
        max_size_mb: Max file size in MB
        model: Gemini model name
        run_evaluation: Whether to run comprehensive evaluation on each result
    
    Returns:
        List of results with metadata and evaluation
    """
    # Query files
    files = query_pdf_files(
        limit=limit, 
        province=province, 
        min_size_kb=min_size_kb,
        max_size_mb=max_size_mb
    )
    
    all_results = []
    successful_count = 0
    failed_count = 0
    
    # Summary statistics
    total_quality_score = 0.0
    total_reports_extracted = 0
    
    for i, file_info in enumerate(files, 1):
        print(f"\n{'='*80}")
        print(f"Processing {i}/{len(files)}: {file_info['path']}")
        print(f"  File: {file_info['file_id']}")
        print(f"  Size: {file_info['size_kb']:.1f} KB ({file_info['size_mb']:.2f} MB)")
        print(f"{'='*80}")
        
        try:
            # Construct Drive URI
            drive_uri = f"https://drive.google.com/uc?export=download&id={file_info['file_id']}"
            
            # Extract
            extraction_result = extract_from_drive_url(drive_uri, model=model)
            
            # Run evaluation if enabled
            evaluations = []
            if run_evaluation and extraction_result:
                print(f"\n📊 Running evaluation...")
                
                for report_idx, report_data in enumerate(extraction_result):
                    eval_result = evaluate_extraction(report_data, report_index=report_idx)
                    evaluations.append(eval_result.to_dict())
                    
                    # Print compact summary
                    status_icon = "✅" if eval_result.is_valid else "❌"
                    print(f"   {status_icon} Report {report_idx + 1}: Quality={eval_result.quality_score:.1%}, "
                          f"Passed={eval_result.checks_passed}/{eval_result.total_checks}")
                    
                    # Update statistics
                    total_quality_score += eval_result.quality_score
                    total_reports_extracted += 1
            
            # Store result
            all_results.append({
                "file_info": file_info,
                "success": True,
                "data": extraction_result,
                "evaluations": evaluations,
                "reports_count": len(extraction_result),
            })
            successful_count += 1
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            
            all_results.append({
                "file_info": file_info,
                "success": False,
                "error": str(e),
                "error_type": type(e).__name__,
            })
            failed_count += 1
    
    # Print batch summary
    print(f"\n{'='*80}")
    print(f"BATCH PROCESSING SUMMARY")
    print(f"{'='*80}")
    print(f"\n📊 Processing Statistics:")
    print(f"   Total Files: {len(files)}")
    print(f"   ✅ Successful: {successful_count}")
    print(f"   ❌ Failed: {failed_count}")
    print(f"   Success Rate: {successful_count/len(files)*100:.1f}%")
    
    if run_evaluation and total_reports_extracted > 0:
        avg_quality = total_quality_score / total_reports_extracted
        print(f"\n📈 Quality Metrics:")
        print(f"   Total Reports Extracted: {total_reports_extracted}")
        print(f"   Average Quality Score: {avg_quality:.1%}")
        
        # Count valid vs invalid reports
        all_evaluations = [e for r in all_results if r.get('success') for e in r.get('evaluations', [])]
        valid_reports = sum(1 for e in all_evaluations if e['is_valid'])
        invalid_reports = len(all_evaluations) - valid_reports
        
        print(f"   Valid Reports: {valid_reports}/{len(all_evaluations)}")
        print(f"   Invalid Reports: {invalid_reports}/{len(all_evaluations)}")
    
    return all_results


# Example: Process 3 small files with evaluation
#batch_results = batch_process_from_bigquery(
#    limit=3,
#    min_size_kb=50.0,
#    max_size_mb=1.0,  # Small files for quick testing
#    model=MODEL_NAME,
#    run_evaluation=True  # Enable comprehensive evaluation
#)

# Show detailed evaluation for first result
if batch_results and batch_results[0].get('success'):
    print(f"\n{'='*80}")
    print(f"DETAILED EVALUATION - First File")
    print(f"{'='*80}")
    
    first_result = batch_results[0]
    if first_result['evaluations']:
        for eval_dict in first_result['evaluations']:
            print(f"\nReport Quality Score: {eval_dict['quality_score']:.1%}")
            print(f"Checks Passed: {eval_dict['checks_passed']}/{eval_dict['total_checks']}")
            print(f"Errors: {len(eval_dict['errors'])}")
            print(f"Warnings: {len(eval_dict['warnings'])}")

NameError: name 'batch_results' is not defined

In [ ]:
# Save batch results with evaluations
if batch_results:
    # Prepare output data
    output_data = {
        "metadata": {
            "model": MODEL_NAME,
            "total_files": len(batch_results),
            "successful": sum(1 for r in batch_results if r.get('success')),
            "failed": sum(1 for r in batch_results if not r.get('success')),
            "total_reports": sum(r.get('reports_count', 0) for r in batch_results if r.get('success')),
        },
        "results": batch_results
    }
    
    # Calculate aggregate metrics
    if any(r.get('evaluations') for r in batch_results):
        all_evals = [e for r in batch_results if r.get('success') for e in r.get('evaluations', [])]
        
        if all_evals:
            output_data["metadata"]["quality_metrics"] = {
                "average_quality_score": sum(e['quality_score'] for e in all_evals) / len(all_evals),
                "valid_reports": sum(1 for e in all_evals if e['is_valid']),
                "invalid_reports": sum(1 for e in all_evals if not e['is_valid']),
                "total_checks_run": sum(e['total_checks'] for e in all_evals),
                "total_passed": sum(e['checks_passed'] for e in all_evals),
                "total_failed": sum(e['checks_failed'] for e in all_evals),
                "total_warnings": sum(e['checks_warning'] for e in all_evals),
            }
    
    # Save to file
    output_file = f"batch_results_{len(batch_results)}files.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)
    
    print(f"\n💾 Batch results saved to: {output_file}")
    
    # Display quality metrics
    if "quality_metrics" in output_data["metadata"]:
        metrics = output_data["metadata"]["quality_metrics"]
        print(f"\n📈 Aggregate Quality Metrics:")
        print(f"   Average Quality Score: {metrics['average_quality_score']:.1%}")
        print(f"   Valid Reports: {metrics['valid_reports']}/{metrics['valid_reports'] + metrics['invalid_reports']}")
        print(f"   Total Checks Run: {metrics['total_checks_run']}")
        print(f"   Overall Pass Rate: {metrics['total_passed']/metrics['total_checks_run']*100:.1f}%")
    
    # Create summary DataFrame
    summary_data = []
    for r in batch_results:
        row = {
            "File": r['file_info']['path'].split('/')[-1] if r.get('file_info') else 'Unknown',
            "Province": r['file_info']['province_name'] if r.get('file_info') else 'N/A',
            "Size (KB)": r['file_info']['size_kb'] if r.get('file_info') else 0,
            "Success": "✅" if r.get('success') else "❌",
            "Reports": r.get('reports_count', 0),
        }
        
        # Add evaluation metrics if available
        if r.get('evaluations'):
            avg_quality = sum(e['quality_score'] for e in r['evaluations']) / len(r['evaluations'])
            row["Quality"] = f"{avg_quality:.1%}"
            row["Valid"] = sum(1 for e in r['evaluations'] if e['is_valid'])
        else:
            row["Quality"] = "N/A"
            row["Valid"] = "N/A"
        
        summary_data.append(row)
    
    print(f"\n📊 Batch Processing Summary Table:")
    df_summary = pd.DataFrame(summary_data)
    display(df_summary)
    
    # Save summary as CSV
    summary_csv = f"batch_summary_{len(batch_results)}files.csv"
    df_summary.to_csv(summary_csv, index=False, encoding='utf-8-sig')
    print(f"\n💾 Summary CSV saved to: {summary_csv}")

## 14. Save Batch Results with Evaluations

## Summary

This notebook demonstrates the **most efficient workflow** for vote extraction:

✅ **BigQuery Integration** - Query metadata to find files  
✅ **Google Drive Direct Access** - No local downloads needed!  
✅ **External URLs Method** - Gemini fetches files directly  
✅ **Structured Output** - Guaranteed JSON schema  
✅ **Pydantic Validation** - Type-safe data models  
✅ **Batch Processing** - Process multiple files efficiently  

## Key Advantages Over Local PDF Processing:

1. **No Local Storage** - Files stay in Google Drive
2. **No PDF Conversion** - Gemini handles PDF directly
3. **Faster** - No download/upload overhead
4. **Scalable** - Easy to process thousands of files
5. **Cost Effective** - No egress charges for data transfer

## Next Steps:

1. **Process by Province** - Filter BigQuery by province
2. **Save to BigQuery** - Store results back in BigQuery
3. **Error Handling** - Add retry logic for failed extractions
4. **Monitoring** - Track processing status and quality metrics
5. **Automation** - Schedule regular batch processing